<a href="https://colab.research.google.com/github/maddogmikeb/Jira/blob/master/TimeInStatus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# install dependencies

from IPython.core.display import clear_output

!pip install -q atlassian-python-api
!pip install -q tqdm

clear_output()

In [32]:
# Configure display

from IPython.core.display import clear_output
from google.colab import data_table
import json
data_table.enable_dataframe_formatter()

def printjson(obj):
  print(json.dumps(obj, indent=2))

clear_output()

In [33]:
# Log in

from IPython.core.display import display, HTML, clear_output
from atlassian import Jira
from google.colab import userdata

jira = Jira(
  url=userdata.get('atlassian_host'),
  username=userdata.get('atlassian_username'),
  password=userdata.get('atlassian_apikey'),
  cloud=True
)

me = jira.myself()

display(HTML('<table><tr><td>' + me["displayName"] + '</td><td><img src="' + me["avatarUrls"]["32x32"] + '"/><td></tr></table>'))


Mike Burns


In [34]:
# Get all issues from jql
# Code to help debug -> https://github.com/atlassian-api/atlassian-python-api/blob/master/atlassian/jira.py

from IPython.display import clear_output, display
from atlassian import Jira
from tqdm import tqdm

def fetch_issues(jql, fields, limit=None):
  params = {
    "fields": fields,
    "jql": jql,
    "expand": "names",
    "maxResults": 100
  }
  if limit is not None:
    params["maxResults"] = int(limit)

  url = jira.resource_url("search")
  start = 0
  results = []
  names = {}

  progress_bar = tqdm(total=0, unit='issue', unit_scale=True, desc='Fetching issues', leave=True, ncols=100)

  while True:
    clear_output(wait=True)
    params["startAt"] = int(start)
    response = jira.get(url, params=params)
    if not response:
      break
    if "names" in response:
      names = response["names"]
    issues = response["issues"]
    results.extend(issues)
    total = int(response["total"])

    progress_bar.total = total
    progress_bar.update(len(issues))

    #display(f"DBG: response: total={total} start={params['startAt']} max={params['maxResults']}")
    if limit is not None or total <= len(response["issues"]) + start:
      break
    start += len(issues)

  clear_output()
  progress_bar.close()
  return results, names

# Define the JQL query
#JQL = 'project in ("Short Stay", "TRACE Program") and (((created >= -500d or updated >= -500d or lastViewed >= -500d) and issuetype not in subTaskIssueTypes() and statusCategory in (Done)) or issuetype in (epic, pattern)) order by created DESC'
# JQL = 'key = FDSEWMSR-17878'
JQL = 'fixversion = "C-R1.1 (Compliance)" and issuetype not in (E-Release)'

fields = "key,summary,created,resolutiondate,resolution,status,issuetype,project,parent,components,fixVersions,customfield_10168,customfield_10001,customfield_10208,customfield_10212,customfield_10032,customfield_10197,customfield_10198,customfield_10209,customfield_10200,statuscategorychangedate"

# Fetch issues using the defined JQL query
issues, names = fetch_issues(JQL, fields)

clear_output()

In [36]:
# Get all the change logs and iterate through them to find all the status changes

# NOTE: This section seems to fail when running as part of a batch / all cells. Just re-run manually and it should work.

from IPython.display import clear_output, display
from atlassian import Jira
import datetime
import pandas as pd
import numpy as np
import copy
from tqdm.contrib.concurrent import process_map
import traceback

def process_changelog(issue):
    changelog = jira.get_issue_changelog(issue["key"])
    changes, flags = [], []
    last_change = issue["fields"]["Created"]
    last_flagged = issue["fields"]["Created"]
    is_flagged = False

    for log in changelog["values"]:
      for logitem in log["items"]:
        if logitem["field"].upper() == "STATUS":
          logitem["start"] = last_change
          logitem["end"] = log["created"]
          last_change = log["created"]
          changes.append({
            'statusid': logitem["from"],
            'status': logitem["fromString"],
            'total': (datetime.datetime.fromisoformat(logitem["end"]) - datetime.datetime.fromisoformat(logitem["start"])).total_seconds()
          })

        if logitem["field"].upper() == "FLAGGED":
          is_flagged = not is_flagged
          logitem["start"] = last_flagged
          logitem["end"] = log["created"]
          last_flagged = log["created"]
          if logitem["fromString"] == "Impediment":
            flags.append({
              'id': logitem["from"],
              'total': (datetime.datetime.fromisoformat(logitem["end"]) - datetime.datetime.fromisoformat(logitem["start"])).total_seconds()
            })

    if len(changes) > 0:
      changes.append({
        'statusid': issue["fields"]["Status"]["id"],
        'status': issue["fields"]["Status"]["name"],
        'total': float('inf')
      })

    changes_df = pd.DataFrame(changes)
    try:
      changes_df.groupby('statusid', as_index=False)['total'].sum()
    except:
      pass
    changes_df = changes_df.reset_index().replace({None: np.nan})

    return changes_df, pd.DataFrame(flags), is_flagged

def process_issue(issue):
  try:
    # Change custom field ids to rendered names
    for key, value in names.items():
      if key in issue["fields"]:
        issue["fields"][value] = issue["fields"].pop(key)

    issue["summary"] = issue["fields"]["Summary"]
    issue["created"] = issue["fields"]["Created"]
    issue["status"] = f"{issue['fields']['Status']['id']}|{issue['fields']['Status']['name']}"
    issue["statusCategory"] = issue["fields"]["Status"]["statusCategory"]["name"]
    issue["resolved"] = issue["fields"]["Resolved"]
    issue["resolution"] = issue["fields"]["Resolution"]["name"] if "Resolution" in issue["fields"] and issue["fields"]["Resolution"] is not None else None
    issue["project"] = issue["fields"]["Project"]["name"]
    issue["team"] = issue["fields"]["Team"]["name"] if "Team" in issue["fields"] and issue["fields"]["Team"] is not None else None
    issue["stream_responsible"] = issue["fields"]["Stream Responsible"]["value"] if "Stream Responsible" in issue["fields"] and issue["fields"]["Stream Responsible"] is not None else None
    issue["components"] = ",".join([component["name"] for component in issue["fields"]["Components"]])
    issue["fixversions"] = ",".join([component["name"] for component in issue["fields"]["Fix versions"]])
    issue["issuetype"] = issue["fields"]["Issue Type"]["name"]
    issue["url"] = jira.url + "browse/" + issue["key"]
    issue["Story Points"] = issue["fields"]["Story Points"]

    issue["Business T-Shirt Size"] = issue["fields"]["Business T-Shirt Size"]["value"] if "Business T-Shirt Size" in issue["fields"] and issue["fields"]["Business T-Shirt Size"] is not None else None
    issue["Business Score"] = issue["fields"]["Business Score"]
    issue["Discovery T-Shirt Estimate"] = issue["fields"]["Discovery T-Shirt Estimate"]["value"] if "Discovery T-Shirt Estimate" in issue["fields"] and issue["fields"]["Discovery T-Shirt Estimate"] is not None else None
    issue["Build T-Shirt Estimate"] = issue["fields"]["Build T-Shirt Estimate"]["value"] if "Build T-Shirt Estimate" in issue["fields"] and issue["fields"]["Build T-Shirt Estimate"] is not None else None
    issue["Program Increment"] = issue["fields"]["Program Increment"]["value"] if "Program Increment" in issue["fields"] and issue["fields"]["Program Increment"] is not None else None

    issue["Status Category Changed"] = issue["fields"]["Status Category Changed"] if "Status Category Changed" in issue["fields"] else None

    # Process changelog

    changes, flags, is_flagged = process_changelog(issue)

    if len(changes) > 0:
      for _, row in changes.iterrows():
        issue[f"{row['statusid']}|{row['status']}"] = row["total"]

    issue["isFlagged"] = is_flagged
    if len(flags) > 0:
      issue["totalFlagged"] = flags['total'].sum()
    else:
      if is_flagged:
        issue["totalFlagged"] = np.inf
      else:
        issue["totalFlagged"] = ''

    issue["parent"], issue["grandparent"] = None, None
    if "Parent" in issue["fields"] and issue["fields"]["Parent"] and "key" in issue["fields"]["Parent"]:
      parent_key = issue["fields"]["Parent"]["key"]
      issue["parent"] = parent_key
      parent_response = jira.issue(parent_key, "parent")
      if parent_response and "fields" in parent_response and "parent" in parent_response["fields"]:
        grandparent_key = parent_response["fields"]["parent"]["key"]
        issue["grandparent"] = grandparent_key

    del issue["fields"], issue["expand"], issue["self"]
    return issue

  except Exception as ex:
    traceback.print_exc()
    raise

# Process issues
processed_issues = process_map(process_issue, copy.deepcopy(issues), max_workers=20, chunksize=1)
#processed_issues = process_issue(copy.deepcopy(issues)[99])
#printjson(processed_issues)

clear_output()

In [38]:
# Print

from IPython.core.display import display, HTML, clear_output
import json
import pandas as pd
import numpy as np

#printjson(processed_issues)

pd.set_option('future.no_silent_downcasting', True)

df = pd.DataFrame(processed_issues)
df = df.reindex(sorted(df.columns, reverse=True), axis=1).replace({None: np.nan, np.inf: np.nan})
# display(df)

df.to_excel("output.xlsx", index=False)